In [5]:
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [6]:
vehicle_images = glob.glob('train_data/vehicles/**/*.png')
non_vehicle_images = glob.glob('train_data/non-vehicles/**/*.png')

In [7]:
def get_spatial_bin_features(img, size=(32, 32)):
    # convert to feature vec
    features = cv2.resize(img, size).ravel() 
    return features

def get_color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if visualize==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm= 'L2-Hys',
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       block_norm= 'L2-Hys',
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features


# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        
        # spatial binning
        if spatial_feat == True:
            spatial_features = get_spatial_bin_features(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        
        # Apply color_hist()
        if hist_feat == True:
            hist_features = get_color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [14]:
orient = 9
pix_per_cell = 8
cell_per_block = 2
size = (16,16)
hist_bins = 32

color_spaces = ['YCrCb','HSV', 'YUV', 'HLS']
channels = ['ALL', 0, 1, 2]

In [17]:
def model(X_train, y_train, X_test, y_test):
    svc = LinearSVC()
    svc.fit(X_train, y_train)
    acc = round(svc.score(X_test, y_test),4)
    return svc, acc

In [19]:
for color_space in color_spaces:
    for hog_channel in channels:
        vehicle_features = extract_features(vehicle_images, color_space=color_space,
                                            spatial_size=size, hist_bins=hist_bins, orient=orient,
                                           pix_per_cell = pix_per_cell, cell_per_block=cell_per_block,
                                           hog_channel=hog_channel)
        non_vehicle_features = extract_features(non_vehicle_images, color_space=color_space,
                                            spatial_size=size, hist_bins=hist_bins, orient=orient,
                                           pix_per_cell = pix_per_cell, cell_per_block=cell_per_block,
                                           hog_channel=hog_channel)

        X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)
        y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))

        X_scaler = StandardScaler().fit(X)
        scaled_X = X_scaler.transform(X)

        X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=101)

        _, accuracy = model(X_train,y_train, X_test, y_test)
        print('CSpace: {}, Channel: {} Test Accuracy: {:.4f}'.format(color_space,hog_channel,accuracy*100))
    print('\n')

CSpace: YCrCb, Channel: ALL Test Accuracy: 98.9000
CSpace: YCrCb, Channel: 0 Test Accuracy: 97.9700
CSpace: YCrCb, Channel: 1 Test Accuracy: 97.1300
CSpace: YCrCb, Channel: 2 Test Accuracy: 96.7900


CSpace: HSV, Channel: ALL Test Accuracy: 98.3700
CSpace: HSV, Channel: 0 Test Accuracy: 97.2100
CSpace: HSV, Channel: 1 Test Accuracy: 97.6900
CSpace: HSV, Channel: 2 Test Accuracy: 97.7200


CSpace: YUV, Channel: ALL Test Accuracy: 98.6800
CSpace: YUV, Channel: 0 Test Accuracy: 97.7800
CSpace: YUV, Channel: 1 Test Accuracy: 96.5900
CSpace: YUV, Channel: 2 Test Accuracy: 97.3800


CSpace: HLS, Channel: ALL Test Accuracy: 97.8300
CSpace: HLS, Channel: 0 Test Accuracy: 96.0300
CSpace: HLS, Channel: 1 Test Accuracy: 97.5500
CSpace: HLS, Channel: 2 Test Accuracy: 96.2800


